In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=db5779af2cd6cbb703a5374387eabc20e211bd00484d01d991d0b5000cb16989
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("basu").getOrCreate()

In [5]:
df=spark.read.csv("/content/EcommerceDataset_Final (1).csv",inferSchema=True,header=True)
df.show()
df.printSchema()

+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+
|    614|     Patricia Smith|   Book|       2|   330.71|2024-09-05 00:00:00|       36.96|         1|   Canada|   Premium|202

In [10]:
from pyspark.sql.functions import *
# Calculate Order Value
df = df.withColumn("OrderValue", round((col("Quantity") * col("UnitPrice")),2))
df.show()

+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|OrderValue|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+
|    614|     Patricia Smith|   Book|       2|   330.71|2024-09-05 00:00:00|       36.96|  

In [15]:

# Calculate total sales by country
total_sales_by_country = df.groupBy('Country').agg(round(sum('TotalSpent').alias('TotalSales'),2))
total_sales_by_country.show()

+---------+---------------------------------------+
|  Country|round(sum(TotalSpent) AS TotalSales, 2)|
+---------+---------------------------------------+
|  Germany|                              851344.44|
|      USA|                             1158384.09|
|       UK|                             1087931.61|
|   Canada|                              889565.57|
|Australia|                              955342.44|
+---------+---------------------------------------+



In [17]:
# Calculate total spent by membership
total_spent_by_membership = df.groupBy('Membership').agg(sum('TotalSpent').alias('TotalSpent'))
total_spent_by_membership.show()

+----------+------------------+
|Membership|        TotalSpent|
+----------+------------------+
|   Premium|        1724985.15|
|       VIP|1549510.0799999991|
|  Standard| 1668072.920000001|
+----------+------------------+



In [19]:
# Calculate sum of Quantity by Product and Country
quantity_sum = df.groupBy('Product', 'Country').agg(sum('Quantity').alias('TotalQuantity'))
# Sort by Product and then by TotalQuantity in descending order
sorted_quantity_sum = quantity_sum.orderBy('Product', 'TotalQuantity', ascending=[True, False])
sorted_quantity_sum.show()

+-------+---------+-------------+
|Product|  Country|TotalQuantity|
+-------+---------+-------------+
|   Bike|       UK|          105|
|   Bike|Australia|          102|
|   Bike|      USA|           92|
|   Bike|   Canada|           75|
|   Bike|  Germany|           59|
|   Book|      USA|          153|
|   Book|  Germany|          138|
|   Book|       UK|          133|
|   Book|Australia|           96|
|   Book|   Canada|           96|
| Laptop|Australia|          127|
| Laptop|      USA|           96|
| Laptop|       UK|           96|
| Laptop|   Canada|           93|
| Laptop|  Germany|           56|
|  Shirt|       UK|          107|
|  Shirt|      USA|          105|
|  Shirt|  Germany|          100|
|  Shirt|Australia|           98|
|  Shirt|   Canada|           76|
+-------+---------+-------------+
only showing top 20 rows



In [24]:

# Calculate sum of Shipping Cost by Product
shipping_cost_sum = df.groupBy('Product') \
                      .agg(round(sum('ShippingCost'),2).alias('TotalShippingCost')) \
                      .orderBy(col('TotalShippingCost').desc())
shipping_cost_sum.show()

+-------+-----------------+
|Product|TotalShippingCost|
+-------+-----------------+
|   Book|         13398.91|
| Laptop|         10159.11|
|   Bike|          9329.49|
|  Shirt|          9197.72|
|    Toy|          8152.25|
+-------+-----------------+



In [26]:
df = df.withColumn('TotalSales', col('OrderValue') + col('ShippingCost'))
df.show()

+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+------------------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|OrderValue|        TotalSales|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+------------------+
|    614|     Patricia Smith|   Bo

In [30]:
# Create or replace a temporary view
df.createOrReplaceTempView("ecommerce_orders")
# Perform SQL query to calculate total sales by EmailDomain
total_sales_by_domain = spark.sql("""
    SELECT
        EmailDomain,
        SUM(OrderValue) AS TotalSales
    FROM
        ecommerce_orders
    GROUP BY
        EmailDomain
    ORDER BY
        TotalSales DESC
""")

total_sales_by_domain.show()

+-------------+------------------+
|  EmailDomain|        TotalSales|
+-------------+------------------+
|    eshop.com| 317918.4199999997|
|shopworld.com|          308844.6|
|  buyeasy.com|303046.55999999965|
|bestdeals.com|263479.92999999993|
|         NULL|          34884.81|
+-------------+------------------+



In [31]:

# Create or replace a temporary view
df.createOrReplaceTempView("ecommerce_orders")

# Perform SQL query to calculate average loyalty period by membership

average_loyalty_period = spark.sql("""
WITH CustomerLoyalty AS (
    SELECT
        CustomerID,
        Membership,
        MIN(SignUpDate) AS SignUpDate,
        MAX(OrderDate) AS LastOrderDate
    FROM
        ecommerce_orders
    GROUP BY
        CustomerID, Membership
)

SELECT
    Membership,
    AVG(DATEDIFF(LastOrderDate, SignUpDate)) AS AverageLoyaltyPeriod
FROM
    CustomerLoyalty
GROUP BY
    Membership
ORDER BY
    AverageLoyaltyPeriod DESC;
""")
average_loyalty_period.show()

+----------+--------------------+
|Membership|AverageLoyaltyPeriod|
+----------+--------------------+
|  Standard|   597.1151832460733|
|       VIP|   567.1256830601093|
|   Premium|   555.9354838709677|
+----------+--------------------+



In [ ]:
# This is a Common Table Expression (CTE) named CustomerLoyalty. It performs a subquery to calculate the minimum SignUpDate and maximum OrderDate for each customer, grouped by CustomerID and Membership.
# The purpose of this CTE is to find the earliest sign-up date and the latest order date for each unique combination of customer and membership type. This will help us calculate the loyalty period for each customer.

In [36]:

# Create or replace a temporary view
df.createOrReplaceTempView("ecommerce_orders")

# Perform SQL query to get top 5 customers by spending
result = spark.sql("""
WITH CustomerSpending AS (
    SELECT
        CustomerID,
        SUM(OrderValue) AS TotalSpent,
        COUNT(OrderID) AS OrderCount,
        AVG(OrderValue) AS AverageOrderValue
    FROM
        ecommerce_orders
    GROUP BY
        CustomerID
),

CustomerLoyalty AS (
    SELECT
        CustomerID,
        MIN(SignUpDate) AS SignUpDate,
        MAX(OrderDate) AS LastOrderDate,
        DATEDIFF(MAX(OrderDate), MIN(SignUpDate)) AS LoyaltyPeriod
    FROM
        ecommerce_orders
    GROUP BY
        CustomerID
),

TopCustomers AS (
    SELECT
        cs.CustomerID,
        cs.TotalSpent,
        cs.OrderCount,
        cs.AverageOrderValue,
        cl.LoyaltyPeriod
    FROM
        CustomerSpending cs
    JOIN
        CustomerLoyalty cl ON cs.CustomerID = cl.CustomerID
    WHERE
        cs.TotalSpent > 1000  -- Filter for customers who spent more than $1,000
)

SELECT
    e.CustomerName,
    t.CustomerID,
    t.TotalSpent,
    t.OrderCount,
    t.AverageOrderValue,
    t.LoyaltyPeriod
FROM
    TopCustomers t
JOIN
    ecommerce_orders e ON t.CustomerID = e.CustomerID
ORDER BY
    t.TotalSpent DESC
LIMIT 5;
""")
result.show()

+----------------+----------+----------+----------+------------------+-------------+
|    CustomerName|CustomerID|TotalSpent|OrderCount| AverageOrderValue|LoyaltyPeriod|
+----------------+----------+----------+----------+------------------+-------------+
|Elizabeth Miller|       154|  10169.36|         3| 3389.786666666667|          782|
|   Richard Brown|       154|  10169.36|         3| 3389.786666666667|          782|
|  Patricia Lopez|       154|  10169.36|         3| 3389.786666666667|          782|
|   Jessica Jones|       654|   9565.05|         5|1913.0099999999998|          675|
|  Jennifer Smith|       654|   9565.05|         5|1913.0099999999998|          675|
+----------------+----------+----------+----------+------------------+-------------+



In [39]:
df.createOrReplaceTempView("ecommerce_orders")

# Perform SQL query to analyze customer spending and loyalty
result = spark.sql("""
WITH CustomerMetrics AS (
    SELECT
        CustomerID,
        SUM(OrderValue) AS TotalSpent,
        COUNT(OrderID) AS OrderCount,
        AVG(OrderValue) AS AverageOrderValue,
        MIN(SignUpDate) AS SignUpDate,
        MAX(OrderDate) AS LastOrderDate,
        DATEDIFF(MAX(OrderDate), MIN(SignUpDate)) AS LoyaltyPeriod
    FROM
        ecommerce_orders
    GROUP BY
        CustomerID
)

SELECT
    cm.CustomerID,
    MAX(e.CustomerName) AS CustomerName,  -- Use MAX to get a single name per customer
    cm.TotalSpent,
    cm.OrderCount,
    cm.AverageOrderValue,
    cm.LoyaltyPeriod
FROM
    CustomerMetrics cm
JOIN
    ecommerce_orders e ON cm.CustomerID = e.CustomerID
WHERE
    cm.TotalSpent > 1000  -- Filter for customers who spent more than $1,000
GROUP BY
    cm.CustomerID, cm.TotalSpent, cm.OrderCount, cm.AverageOrderValue, cm.LoyaltyPeriod
ORDER BY
    cm.TotalSpent DESC
LIMIT 5;
""")
result.show()

+----------+--------------+----------+----------+------------------+-------------+
|CustomerID|  CustomerName|TotalSpent|OrderCount| AverageOrderValue|LoyaltyPeriod|
+----------+--------------+----------+----------+------------------+-------------+
|       154| Richard Brown|  10169.36|         3| 3389.786666666667|          782|
|       654|Thomas Johnson|   9565.05|         5|1913.0099999999998|          675|
|       465| William Davis|    8732.7|         4|          2183.175|          714|
|       256|Sarah Anderson|    8706.4|         3| 2902.133333333333|         1012|
|       726|Richard Martin|    8213.4|         5|1642.6799999999998|          380|
+----------+--------------+----------+----------+------------------+-------------+



In [40]:
df.show()

+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+------------------+
|OrderID|       CustomerName|Product|Quantity|UnitPrice|          OrderDate|ShippingCost|CustomerID|  Country|Membership|         SignUpDate|      LastOrderDate|TotalSpent|Customer Communication Log|   CommunicationDate|         InquiryType|  CommunicationIssue| EmailUsername|  EmailDomain|OrderValue|        TotalSales|
+-------+-------------------+-------+--------+---------+-------------------+------------+----------+---------+----------+-------------------+-------------------+----------+--------------------------+--------------------+--------------------+--------------------+--------------+-------------+----------+------------------+
|    614|     Patricia Smith|   Bo

In [43]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [51]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Step 2: Load the dataset
df = pd.read_csv("/content/EcommerceDataset_Final (1).csv")

# Step 3: Define features and target variable
X = df[['Quantity', 'UnitPrice', 'ShippingCost']]  # Features
y = df['TotalSpent']  # Target variable

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Create a linear regression model
model = LinearRegression()

# Step 6: Fit the model to the training data
model.fit(X_train, y_train)

# Step 7: Make predictions on the test data
y_pred = model.predict(X_test)
y_pred
# Step 8: Evaluate the model
# mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
# rmse = np.sqrt(mse)  # Root Mean Squared Error
# r2 = r2_score(y_test, y_pred)  # R² Score

# # Step 9: Print the model coefficients and evaluation metrics
# print("Coefficients:", model.coef_)
# print("Intercept:", model.intercept_)
# print("Mean Squared Error (MSE):", mse)
# print("Root Mean Squared Error (RMSE):", rmse)
# print("R² Score:", r2)

# # Step 10: Plotting the results
# plt.scatter(y_test, y_pred)
# plt.xlabel("Actual Total Spent")
# plt.ylabel("Predicted Total Spent")
# plt.title("Actual vs Predicted Total Spent")
# plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)  # Line for perfect prediction
# plt.show()

array([4944.77773067, 5045.89816518, 4863.26033062, 5193.10978201,
       4878.01667045, 4957.04874433, 4921.40917518, 5103.75607212,
       4771.29194251, 5019.43740969, 5182.17566339, 4732.379309  ,
       5051.20329704, 4884.61068003, 4910.77260869, 4906.17976761,
       4916.76950963, 5042.48514263, 5197.46264446, 4991.27800193,
       4984.04041788, 5083.44337118, 5245.06070123, 5137.508452  ,
       4918.11730611, 4936.10408326, 4915.80890222, 5055.77319308,
       4753.25817304, 5197.95592197, 4804.71372999, 5004.27473866,
       5246.79403696, 4926.15881523, 4763.80960728, 4964.08049187,
       4859.74944659, 4804.23275783, 4978.17710738, 5159.2226988 ,
       5024.05344924, 5236.31885582, 5051.43823619, 5016.91323158,
       5178.97744496, 5287.41312556, 5020.78224735, 4972.57950282,
       5183.7571239 , 5291.38296194, 5022.88074173, 5141.33250901,
       5123.27407812, 5170.20530729, 5215.45821082, 4847.81604672,
       5176.87610254, 4821.73460793, 5000.50497852, 4965.80839